In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
BeautifulSoup	 DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 
get_column_descriptions	 load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 re	 
save_dataframes	 store_objects	 sys	 urllib	 



# Scraping the Availability Stats

In [2]:

wiki_url = 'https://ageofempires.fandom.com/wiki/'
summary_box_url = '{}Age_of_Empires_II:_The_Age_of_Kings'.format(wiki_url)
print(summary_box_url)
with urllib.request.urlopen(summary_box_url) as response:
    summary_box_html = response.read()
summary_box_soup = BeautifulSoup(summary_box_html, 'html.parser')
content_text_selector = '#mw-content-text'
content_text_div = summary_box_soup.select(content_text_selector)[0]
children_list = list(content_text_div.children)
for child in children_list:
    if child.name == 'table':
        summary_box_table = child

https://ageofempires.fandom.com/wiki/Age_of_Empires_II:_The_Age_of_Kings


In [3]:

tr_soup_dict = {}
tr_list = []
th_list = ['The Age of Kings', 'Infantry', 'Archers', 'Cavalry', 'Siege', 'Navy', 'Unique',
           'The Conquerors', 'The Forgotten', 'The African Kingdoms', 'Rise of the Rajas']
military_units_list = ['Infantry', 'Archers', 'Cavalry', 'Siege', 'Navy', 'Unique']
heading_list = []
for summary_box_th in summary_box_table.find_all('th'):
    for heading in th_list:
        if heading in summary_box_th.text:
            tr_soup_dict[heading] = summary_box_th.parent
            if heading in military_units_list:
                tr_list.append(tr_soup_dict[heading])
            else:
                tr_list.append(tr_soup_dict[heading].next_sibling)
            heading_list.append(heading)

In [4]:

def get_link_html(link):
    link_url = 'https://ageofempires.fandom.com{}'.format(link['href'])
    if '_(Age_of_Empires_II)' in link_url:
        with urllib.request.urlopen(link_url) as response:
            link_html = response.read()
    else:
        try:
            extended_link_url = '{}_(Age_of_Empires_II)'.format(link_url)
            with urllib.request.urlopen(extended_link_url) as response:
                link_html = response.read()
            link_url = extended_link_url
        except:
            with urllib.request.urlopen(link_url) as response:
                link_html = response.read()
    
    return link_url, link_html

In [5]:

def process_all_except(appl_civ_text, row_dict):
    for civ_name in civ_name_list:
        if civ_name in appl_civ_text:
            row_dict[civ_name] = 0
        else:
            row_dict[civ_name] = 1
    
    return row_dict

In [6]:

def process_article(art_table_list, row_dict):
    art_table = art_table_list[0]
    art_ul_list = art_table.find_all('ul')
    assert len(art_ul_list) == 2

    # Get available civs
    available_ul = art_ul_list[0]
    for link in available_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 1

    # Get unavailable civs
    unavailable_ul = art_ul_list[1]
    for link in unavailable_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 0

    return row_dict

In [7]:

space_regex = re.compile(r'\s+')
def process_small_list_of_units(appl_civ_text, row_dict):
    appl_civ_list = space_regex.split(appl_civ_text)
    for civ_name in civ_name_list:
        row_dict[civ_name] = 0
    for civ_name in appl_civ_list:
        if civ_name in civ_name_list:
            row_dict[civ_name] = 1
    
    return row_dict

In [8]:

toc_selector = '#toc'
def process_exceptions(link_soup, row_dict):
    desc_p_list = link_soup.select(toc_selector)
    if len(desc_p_list):
        for sib in desc_p_list[0].previous_siblings:
            if 'available to all civilizations except for' in str(sib):
                break
        process_all_except(sib.text.strip(), row_dict)
    
    return row_dict

In [9]:

article_selector = '#mw-content-text > table.article-table'
def process_unit(link_soup, appl_civ_text, row_dict):
    for column_name in civ_name_list:
        columns_set.add(column_name)
    if 'All civilizations' in appl_civ_text:
        for civ_name in civ_name_list:
            row_dict[civ_name] = 1
    elif 'All except ' in appl_civ_text:
        row_dict = process_all_except(appl_civ_text, row_dict)
    elif 'See ' in appl_civ_text:
        art_table_list = link_soup.select(article_selector)
        if len(art_table_list):
            row_dict = process_article(art_table_list, row_dict)
        else:
            row_dict = process_exceptions(link_soup, row_dict)
    else:
        row_dict = process_small_list_of_units(appl_civ_text, row_dict)
    
    return row_dict

In [10]:

aside_columns_dict = load_object('building_columns_dict')
aside_columns_dict

{'A': 'Age', 'HP': 'Hit points', 'Ga': 'Garrison capacity', 'Ar': 'Armor', 'PA': 'Pierce armor', 'LOS': 'Line of Sight', 'Ra': 'Range', 'At': 'Attack', 'W': 'Wood cost', 'S': 'Stone cost', 'G': 'Gold cost', 'CT': 'Construction time in seconds', 'Sz': 'Size in tiles', 'Type': 'Type', 'Intro': 'Introduced in', 'Available': 'Civilization', 'Age': 'Age', 'Time': 'Construction time', 'Use': 'Use', 'Wood': 'Wood', 'Stone': 'Stone', 'Size': 'Size', 'Garrison': 'Garrison', 'Attack': 'Attack', 'BAttack': 'Attack bonuses', 'ROF': 'Rate of Fire', 'Range': 'Range', 'Accuracy': 'Accuracy', 'PSpeed': 'Projectile speed', 'Armor': 'Melee armor', 'PArmor': 'Pierce armor', 'Class': 'Armor classes', 'Required': 'Required building', 'Next': 'Required for', 'MinRange': 'Minimum range', 'UpTo': 'Upgraded to', 'UpCost': 'Upgrade cost', 'UpTime': 'Upgrade time', 'UpFrom': 'Upgraded from', 'Gold': 'Gold', 'Research': 'Required research'}

In [11]:

def consume_aside(unit_soup, row_dict):
    aside_list = unit_soup.select('#mw-content-text > aside')
    for aside_soup in aside_list:
        for data_source_div in aside_soup.find_all('div', {'data-source': True}):
            data_source_h3_list = data_source_div.find_all('h3')
            if len(data_source_h3_list):
                h3_siblings_list = list(data_source_h3_list[0].next_siblings)
                if len(h3_siblings_list):
                    column_name = data_source_div['data-source']
                    full_name = data_source_div.find_all('h3')[0]
                    link_list = full_name.find_all('a')
                    if len(link_list):
                        full_name = link_list[0].text
                    else:
                        full_name = full_name.text
                    aside_columns_dict[column_name] = full_name
                    value_list = list(h3_siblings_list[1].strings)
                    column_value = ' '.join(x.strip() for x in value_list)
                    if column_name == 'Civilization':
                        row_dict = process_unit(unit_soup, column_value, row_dict)
                    else:
                        columns_set.add(column_name)
                        row_dict[column_name] = column_value
    
    return row_dict

In [12]:

appl_civ_selector = '#mw-content-text > aside > section > div[data-source="Civilization"] > div'
title_selector = '#PageHeader > div.page-header__main > h1'
def get_unit_pages(soup, heading):
    for link in soup.find_all('a'):
        unit_row_dict = {'Heading': heading}
        unit_url, unit_html = get_link_html(link)
        unit_soup = BeautifulSoup(unit_html, 'html.parser')
        unit_title = unit_soup.select(title_selector)[0].string.split('(')[0].strip()
        index_list.append(unit_title)
        rows_list.append(consume_aside(unit_soup, unit_row_dict))

In [13]:

civ_name_list = sorted(load_object('civ_list'))
rows_list = []
index_list = []
columns_set = set(['Heading'])
for soup, heading in zip(tr_list, heading_list):
    get_unit_pages(soup, heading)

In [14]:

store_objects(aside_columns_dict=aside_columns_dict)
aside_columns_dict

Pickling to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\aside_columns_dict.pickle


{'A': 'Age', 'HP': 'Hit points', 'Ga': 'Garrison capacity', 'Ar': 'Armor', 'PA': 'Pierce armor', 'LOS': 'Line of Sight', 'Ra': 'Range', 'At': 'Attack', 'W': 'Wood cost', 'S': 'Stone cost', 'G': 'Gold cost', 'CT': 'Construction time in seconds', 'Sz': 'Size in tiles', 'Type': 'Type', 'Intro': 'Introduced in', 'Available': 'Civilization', 'Age': 'Age', 'Time': 'Construction time', 'Use': 'Use', 'Wood': 'Wood', 'Stone': 'Stone', 'Size': 'Size', 'Garrison': 'Garrison', 'Attack': 'Attack', 'BAttack': 'Attack bonuses', 'ROF': 'Rate of Fire', 'Range': 'Range', 'Accuracy': 'Accuracy', 'PSpeed': 'Projectile speed', 'Armor': 'Melee armor', 'PArmor': 'Pierce armor', 'Class': 'Armor classes', 'Required': 'Required research', 'Next': 'Required for', 'MinRange': 'Minimum range', 'UpTo': 'Upgrades to', 'UpCost': 'Upgrade cost', 'UpTime': 'Upgrade time', 'UpFrom': 'Upgraded from', 'Gold': 'Gold', 'Research': 'Required research', 'Civilization': 'Civilization', 'Building': 'Trained at', 'TrainTime': 'T

In [13]:

columns_set

{'Gold', 'Heading', 'Indians', 'Portuguese', 'Japanese', 'FD', 'UpTo', 'Age', 'Mongols', 'Magyars', 'UpCost', 'Teutons', 'Burmese', 'Turks', 'AOE', 'Franks', 'ROF', 'Malians', 'Type', 'Spanish', 'Class', 'Celts', 'Saracens', 'Attack', 'Wood', 'Range', 'HP', 'Incas', 'Koreans', 'Ethiopians', 'Huns', 'LOS', 'Chinese', 'TrainTime', 'MinRange', 'Slavs', 'Armor', 'AttackBonus', 'Vietnamese', 'PierceArmor', 'Intro', 'Aztecs', 'Garrison', 'Accuracy', 'Required', 'Mayans', 'Vikings', 'Berbers', 'Goths', 'Malay', 'Building', 'Khmer', 'Byzantines', 'Persians', 'Food', 'UpFrom', 'UpTime', 'PSpeed', 'Britons', 'Italians', 'Speed'}

In [14]:

civ_name_list

['Aztecs', 'Berbers', 'Britons', 'Burmese', 'Byzantines', 'Celts', 'Chinese', 'Ethiopians', 'Franks', 'Goths', 'Huns', 'Incas', 'Indians', 'Italians', 'Japanese', 'Khmer', 'Koreans', 'Magyars', 'Malay', 'Malians', 'Mayans', 'Mongols', 'Persians', 'Portuguese', 'Saracens', 'Slavs', 'Spanish', 'Teutons', 'Turks', 'Vietnamese', 'Vikings']

In [ ]:

columns_list = ['Heading', 'Class', 'Speed', 'TrainTime', 'Gold', 'UpTime', 'AttackBonus', 'AOE',
                'Armor', 'Garrison', 'LOS', 'Range', 'Accuracy', 'Wood', 'Intro', 'Building', 'ROF',
                'FD', 'UpFrom', 'PierceArmor', 'Age', 'Food', 'Attack', 'MinRange', 'Required',
                'Type', 'UpCost', 'PSpeed', 'UpTo', 'HP', 'Aztecs', 'Berbers', 'Britons', 'Burmese',
                'Byzantines', 'Celts', 'Chinese', 'Ethiopians', 'Franks', 'Goths', 'Huns', 'Incas',
                'Indians', 'Italians', 'Japanese', 'Khmer', 'Koreans', 'Magyars', 'Malay',
                'Malians', 'Mayans', 'Mongols', 'Persians', 'Portuguese', 'Saracens', 'Slavs',
                'Spanish', 'Teutons', 'Turks', 'Vietnamese', 'Vikings']

In [15]:

index_name = 'Unit Name'
other_columns_set = columns_set - set(civ_name_list)
other_columns_set = other_columns_set - set(['Heading'])
columns_list = ['Heading'] + sorted(other_columns_set) + civ_name_list
unit_availability_df = pd.DataFrame(data=rows_list, index=index_list, columns=columns_list)
unit_availability_df.index.name = index_name

In [52]:

columns_list = unit_availability_df.columns
file_path = os.path.join(DATA_CSV_FOLDER, 'unit_availability_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_csv(file_path)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df[columns_list])
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_excel(file_path)

Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\csv\unit_availability_df.csv
Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\unit_availability_df.csv
Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\xlsx\unit_availability_df.xlsx


In [53]:

columns_list

Index(['Heading', 'AOE', 'Accuracy', 'Age', 'Armor', 'Attack', 'AttackBonus',
       'Building', 'Class', 'FD', 'Food', 'Garrison', 'Gold', 'HP', 'Intro',
       'LOS', 'MinRange', 'PSpeed', 'PierceArmor', 'ROF', 'Range', 'Required',
       'Speed', 'TrainTime', 'Type', 'UpCost', 'UpFrom', 'UpTime', 'UpTo',
       'Wood', 'Aztecs', 'Berbers', 'Britons', 'Burmese', 'Byzantines',
       'Celts', 'Chinese', 'Ethiopians', 'Franks', 'Goths', 'Huns', 'Incas',
       'Indians', 'Italians', 'Japanese', 'Khmer', 'Koreans', 'Magyars',
       'Malay', 'Malians', 'Mayans', 'Mongols', 'Persians', 'Portuguese',
       'Saracens', 'Slavs', 'Spanish', 'Teutons', 'Turks', 'Vietnamese',
       'Vikings'],
      dtype='object')